In [5]:
pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import psycopg2
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

22/11/25 03:52:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [18]:
dataframe=spark.read.option("header",True).csv("data.csv")

dataframe.write.option("header",True).partitionBy(["vendor", "bus"]).mode("overwrite").csv("resultados")

DataFrame[bus: string, fechahora: string, lat: string, lng: string, velocidad: string, pasajeros: string, geom: string, vendor: string, id: string]


In [ ]:
def get_Bus_Cant_Prom_Por_Paredero():#Frecuenica
    Hour_list_from = ['2018-06-13 00:00:00+00','2018-06-13 01:00:00+00','2018-06-13 02:00:00+00','2018-06-13 03:00:00+00','2018-06-13 04:00:00+00','2018-06-13 05:00:00+00','2018-06-13 06:00:00+00','2018-06-13 07:00:00+00','2018-06-13 08:00:00+00','2018-06-13 09:00:00+00','2018-06-13 10:00:00+00','2018-06-13 11:00:00+00','2018-06-13 12:00:00+00','2018-06-13 13:00:00+00','2018-06-13 14:00:00+00','2018-06-13 15:00:00+00','2018-06-13 16:00:00+00','2018-06-13 18:00:00+00','2018-06-13 19:00:00+00','2018-06-13 20:00:00+00','2018-06-13 21:00:00+00','2018-06-13 22:00:00+00','2018-06-13 23:00:00+00']

    Hour_list_to = ['2018-06-13 00:59:59+00','2018-06-13 01:59:59+00','2018-06-13 02:59:59+00','2018-06-13 03:59:59+00','2018-06-13 04:59:59+00','2018-06-13 05:59:59+00','2018-06-13 06:59:59+00','2018-06-13 07:59:59+00','2018-06-13 08:59:59+00','2018-06-13 09:59:59+00','2018-06-13 10:59:59+00','2018-06-13 11:59:59+00','2018-06-13 12:59:59+00','2018-06-13 13:59:59+00','2018-06-13 14:59:59+00','2018-06-13 15:59:59+00','2018-06-13 16:59:59+00','2018-06-13 18:59:59+00','2018-06-13 19:59:59+00','2018-06-13 20:59:59+00','2018-06-13 21:59:59+00','2018-06-13 22:59:59+00','2018-06-13 23:59:59+00']
    #*************************
    #print(type(result.index))
    #print(result.index.names)
    #print(result.index.values)
    #print(result.index.levels)

    dataframe_ruta = pd.read_csv("./E04I.csv")
    temp2 = dataframe_ruta.groupby(['parader'], group_keys=True, as_index=True).apply(lambda x: x)
    #temp4 = dataframe_ruta.groupby(['parder_geom'], group_keys=True, as_index=True).apply(lambda x: x)
    list_paredors = temp2.index.levels[0]
    #temp3 = temp2.loc['PA146']
    #a = temp2.loc['PA146']['fechahora'].tolist()[0]

    dataframe_buses = {'paradero':[],'promedio_bus_hora':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[]}
    #print(len(Hour_list_from))
    #print(len(Hour_list_to))

    for i in list_paredors: 
        x = temp2.loc[i]
        List_total_buses = list()
        geo = x['parder_geom'].iat[0]
        geo_lat = x['parder_lat'].iat[0]
        geo_lon = x['parder_lon'].iat[0]
        paradero_list = list()
        #print(geo)
        for j in range(23):
            #print(j)
            df = x[(x['fechahora'] >= Hour_list_from[j]) & (x['fechahora'] <= Hour_list_to[j])]
            List_total_buses.append(df['bus'].nunique())
            #Lista_interval.append(Hour_list_from[j]+' to '+Hour_list_to[j])
        mean = (sum(List_total_buses) / len(List_total_buses))
        dataframe_buses['paradero'].append(i)
        dataframe_buses['promedio_bus_hora'].append(mean)
        dataframe_buses['geo_parader'].append(geo)
        dataframe_buses['geo_parader_lat'].append(geo_lat)
        dataframe_buses['geo_parader_lon'].append(geo_lon)
        if mean < 5:
            dataframe_buses['calificacion'].append(0)
        elif mean > 6:
            dataframe_buses['calificacion'].append(1)
        else:
            dataframe_buses['calificacion'].append(2)


        
        #dataframe_buses['rango_hora'] += Lista_interval


    dataframe_total_busparadero = pd.DataFrame(dataframe_buses)
    print(dataframe_total_busparadero)
    dataframe_total_busparadero.to_csv('E04I_NOTED.csv')